<a href="https://colab.research.google.com/github/Machine-Learning-Tokyo/DL-workshop-series/blob/master/Simple_GAN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [0]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

### Import the necessary functions
This cell will import the predefined functions needed for the execution of the notebook.

If you want to call a function that is defined in this notebook, delete the headed underscore from the function's name

In [0]:
%%capture
%rm -r adf93f9900d1dcfb6d76c83adec74a85
!git clone https://gist.github.com/dkatsios/adf93f9900d1dcfb6d76c83adec74a85.git
!pip install import_ipynb
import import_ipynb
%cd /content/adf93f9900d1dcfb6d76c83adec74a85
from simple_gan_gist import *
%cd /content

### Function to build the generator

In [0]:
def _build_generator(noise_size, img_shape):
  """
  function that takes as input
  the noise_size (integer) and the img_shape (tuple of integers)
  and returns a keras Model.
  The model has 3 blocks of Dense, BatchNormalization and LeakyReLU layers.
  The units at the Dense layers are 256, 512 and 1024 respectively.
  The alpha parameter at the LeakyReLU layers is 0.2.
  The activation of the last layer is tanh.
  The model (generator) takes as input a tensor of shape (noise_size,)
  and returns a tensor of shape img_shape.
  """
  
  noise = Input((noise_size,))
  
  x = Dense(256)(noise)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  # TODO add 2 more blocks at the model
  
  x = Dense(np.prod(img_shape), activation='tanh')(x)
  img = Reshape(img_shape)(x)
  
  generator = Model(noise, img)
  return generator

### Function to build the discriminator

In [0]:
def _build_discriminator(img_shape):
  """
  function that takes as input the img_shape (tuple of integers)
  and returns a keras Model.
  The model has 3 blocks of Dense and LeakyReLU layers.
  The units at the Dense layers are 1024, 512, and 256 respectively.
  The alpha parameter at the LeakyReLU layers is 0.2
  The activation of the last layer is sigmoid.
  The model (discriminator) takes as input a tensor of shape img_shape
  and returns a tensor of shape 1
  """
  
  img = Input(img_shape)
  x = Flatten()(img)
  
  x = Dense(1024)(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  # TODO add 2 more blocks at the model
  
  validity = Dense(1, activation='sigmoid')(x)
  
  discriminator = Model(img, validity)
  return discriminator

### Function to compile the models

In [0]:
def _get_compiled_models(generator, discriminator, noise_size):
  """
  function that takes as input
  the generator (keras.Model)
  the discriminator (keras.Model)
  and the noise_size (integer)
  and return the generator, the compiled discriminator and the compiled comnbined models.
  The combined model takes as input noise (tensor of shape (noise_size,))
  and outputs the validity (output of the discriminator)
  of the internally generated image (output of the generator).
  For both models the optimizer is Adam with learning rate 0.0002 and beta_1 0.5
  and the loss function is binary_crossentropy.
  The discriminator has accuracy as metric.
  """
  
  optimizer = Adam(0.0002, 0.5)
  
  discriminator.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  discriminator.trainable = False
  
  noise = Input((noise_size,))
  img = generator(noise)
  validity = discriminator(img)
  
  # TODO create and compile the combined model
  
  return generator, discriminator, combined

### Function to sample and save generated images

In [0]:
def _sample_imgs(generator, noise_size, step):
  """
  function that takes as input the generator (keras.Model), the noise_size (integer)
  and the step (integer) and generates and saves samples of the generated images.
  The images are in a 5x5 grid and are saved at ./images/{step}.png
  """
  
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r*c, noise_size))
  imgs = generator.predict(noise)
  imgs = imgs / 2 + 0.5
  
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i, j].imshow(imgs[cnt], cmap='gray')
      axs[i, j].axis('off')
      cnt += 1
  fig.savefig('./images/%d.png' % step)
  plt.close()

### Function to train the models

In [0]:
def _train(models, noise_size, img_shape, batch_size, steps):
  """
  function that takes as input the models (tuple of generator, discriminator, combined),
  the noise_size, the img_shape, the batch_size and the steps and trains the models
  for this number of steps on batches of size batch_size.
  The training data are from mnist (keras.datasets).
  For preprocessing, the data are normalized in [-1, 1] (original in [0, 255]).
  Every 100 steps the models loss and accuracy is printed and samples are saved.
  """
  generator, discriminator, combined = models
  # get real data
  (X_train, _), (X_val, _) = mnist.load_data()
  
  # TODO concatenate and normalize mnist images
  
  for step in range(1, steps + 1):
    # train discriminator
    inds = np.random.randint(0, mnist_imgs.shape[0], batch_size)
    real_imgs = mnist_imgs[inds]
    real_validity = np.ones(batch_size)
    
    noise = np.random.normal(0, 1, (batch_size, noise_size))
    gen_imgs = generator.predict(noise)
    gen_validity = np.zeros(batch_size)
    
    r_loss = discriminator.train_on_batch(real_imgs, real_validity)
    g_loss = discriminator.train_on_batch(gen_imgs, gen_validity)
    disc_loss = np.add(r_loss, g_loss) / 2
    
    # train generator
    noise = np.random.normal(0, 1, (batch_size, noise_size))
    gen_validity = np.ones(batch_size)
    
    # TODO train the combined model
    
    #print progress
    if step % 100 == 0:
      print('step: %d, D_loss: %f, D_accuracy: %.2f%%, G_loss: %f' % (step, disc_loss[0],
                                                                      disc_loss[1] * 100, gen_loss))
    
    # save_samples
    if step % 100 == 0:
      sample_imgs(generator, noise_size, step)

### Define hyperparameters

In [0]:
%rm -r /content/images
%mkdir /content/images
noise_size = 100
img_shape = 28, 28
batch_size = 64
steps = 1000

### Generate the models

In [0]:
generator = build_generator(noise_size, img_shape)
discriminator = build_discriminator(img_shape)
compiled_models = get_compiled_models(generator, discriminator, noise_size)

### Train the models

In [0]:
train(compiled_models, noise_size, img_shape, batch_size, steps)

### Display samples

In [0]:
Image('/content/images/%d.png' % 1000)